In [1]:
import getpass
import os


def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")


_set_if_undefined("OPENAI_API_KEY")
_set_if_undefined("LANGCHAIN_API_KEY")
_set_if_undefined("TAVILY_API_KEY")

# Optional, add tracing in LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Email Sending Tool"

## Tools

In [7]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.tools import BaseTool
from typing import Annotated, Type, List
from pydantic import EmailStr
import resend

class MailArgs(BaseModel):
    """
    Input for the email sender tool
    """
    subject: str = Field(..., description="The subject of the email")
    content: str = Field(..., description="The content of the email in HTML format")
    recipients: List[str] = Field(..., description="The list of recipients")

    
class EmailSender(BaseTool):
    """
    A tool for sending emails to a list of recipients.
    """
    name: str = "EmailSender"
    description: str = (
        "A tool for sending emails to a list of recipients."
        )
    args_schema: Type[BaseModel] = MailArgs

    def _run(
            self,
            subject: str,
            content: str,
            recipients: List[str]
            ):
        """
        Use the tool
        """
        try:
            resend.api_key = "re_DMzZbNtC_EALxDZQYCGM7f46wnegaRdbV"
            params: resend.Emails.SendParams = {
                "from": "Acme <onboarding@resend.dev>",
                "to": recipients,
                "subject": subject,
                "html": content,
            }
            resend.Emails.send(params)
            return "Email sent successfully"
        except Exception as e:
            return repr(e)
        
        


email_sender_tool = EmailSender()



In [8]:
# Example usage of the email_sender_tool
result = email_sender_tool.run({
    "subject": "Test Email",
    "content": "<h1>Hello!</h1><p>This is a test email sent using the EmailSender tool.</p>",
    "recipients": ["ibrahim.aka.ajax@gmail.com"]
})

print(result)

Email sent successfully
